In [1]:
from os.path import join, exists
from os import makedirs, listdir
import pandas as pd
import random
import numpy as np
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
import torch
from torch import nn, optim
from model.dataloader import DailyTimeSeriesFromPandas
from model.networks import *

seed=1234

# For reproductibility
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)

from sys import path
path.append('/home/clement/Documents/phd/JuNNo/lib/')
from junno.j_utils import log, Process
log

<IPython.core.display.Javascript object>

In [2]:
INPUT = "../data/"
input_data = pd.read_pickle(join(INPUT, "input"))
#cs_s1 and cs_s2 indicates day and night, threshold at 5 MegaWatt q3/4 in s2; wf 

In [3]:
# input_data = input_data.loc[input_data.cspower_s2 > 5] # Keep only day

input_data = input_data.between_time('6:00', '20:00')
input_data.index = pd.DatetimeIndex([i.replace(tzinfo=None) for i in input_data.index]) # Remove timezone from the timeStamp
input_data = input_data.dropna(axis="columns", how="any") # Remove columns that contain NaN
input_data.fillna(0, inplace=True)
input_data

,hcc-a,lcc-a,mcc-a,pr-a,rh-a,sp-a,tcc-a,temp-a,tp-a,uwind-a,...,pr-q4,rh-q4,sp-q4,tcc-q4,temp-q4,tp-q4,uwind-q4,vwind-q4,cspower_s1,cspower_s2
2016-01-01 06:00:00,0.000000,40.136719,51.904297,102061.169434,74.435987,101969.476318,51.904297,268.417603,0.000000,2.402274,...,101870.544434,71.217237,101506.976318,1.367188,266.073853,0.0,1.167899,-1.423027,-0.025516,-0.014530
2016-01-01 06:30:00,0.000000,29.687500,39.941406,102089.868164,74.489554,101994.500732,39.941406,268.325165,0.000000,2.018408,...,101894.555664,71.302054,101532.000732,9.301758,265.989227,0.0,1.155127,-1.426312,-0.025516,-0.014530
2016-01-01 07:00:00,0.000000,19.238281,27.978516,102118.566895,74.543121,102019.525146,27.978516,268.232727,0.000000,1.634542,...,101918.566895,71.386871,101557.025146,17.236328,265.904602,0.0,1.142354,-1.429598,-0.025516,-0.014530
2016-01-01 07:30:00,0.000000,14.062500,24.487305,102158.859253,74.182284,102064.111328,24.487305,268.184807,0.000000,1.388641,...,101949.874878,69.713534,101589.111328,18.920898,266.208244,0.0,1.181610,-1.398719,35.357178,34.007533
2016-01-01 08:00:00,0.000000,8.886719,20.996094,102199.151611,73.821447,102108.697510,20.996094,268.136887,0.000000,1.142741,...,101981.182861,68.040197,101621.197510,20.605469,266.511887,0.0,1.220866,-1.367840,77.497064,51.269383
2016-01-01 08:30:00,0.000000,6.079102,13.793945,102231.594849,69.186267,102138.690186,13.793945,268.991737,0.000000,1.117902,...,102005.422974,64.076892,101644.940186,18.212891,267.405800,0.0,1.488996,-1.300178,100.028277,63.689221
2016-01-01 09:00:00,0.000000,3.271484,6.591797,102264.038086,64.551086,102168.682861,6.591797,269.846588,0.000000,1.093062,...,102029.663086,60.113586,101668.682861,15.820312,268.299713,0.0,1.757125,-1.232515,118.403682,73.794003
2016-01-01 09:30:00,0.000000,1.635742,3.295898,102272.842407,57.628631,102177.966309,3.295898,270.760429,0.000000,1.585017,...,102028.701782,55.394256,101665.466309,10.913086,269.244804,0.0,2.120173,-1.365683,132.996183,81.845500
2016-01-01 10:00:00,0.000000,0.000000,0.000000,102281.646729,50.706175,102187.249756,0.000000,271.674271,0.000000,2.076972,...,102027.740479,50.674925,101662.249756,6.005859,270.189896,0.0,2.483222,-1.498850,144.170659,88.031562
2016-01-01 10:30:00,0.000000,0.000000,0.000000,102259.939575,46.689870,102165.698242,0.000000,272.308136,0.000000,2.256882,...,102003.689575,48.424245,101640.698242,3.320312,270.550323,0.0,2.784226,-1.727962,152.146278,92.436213


In [4]:
output_data = pd.read_pickle(join(INPUT, "output"))
output_data = output_data.loc[input_data.index, :] # Get corresponding groundtruth
if len(output_data)!= len(input_data):
    raise ValueError("Mismatch between input length (%i) and output length (%i)" 
                     %(len(input_data), len(output_data)))
# output_data = output_data.drop(columns='s2')

In [5]:
# Split between train, test and validation (60, 30, 10)
train_size = 0.6
test_size = 0.3
validation_size = max(1-train_size-test_size, 0)

list_days = output_data.index.map(lambda t: t.date()).unique() # Get list of unique days
nb_days = len(list_days)
indexes = np.arange(nb_days)
np.random.shuffle(indexes)

train_days =  list_days[indexes[:int(train_size*nb_days)]]
test_days = list_days[indexes[int(train_size*nb_days):int((test_size+train_size)*nb_days)]]
validation_days = list_days[indexes[int((train_size+test_size)*nb_days):]]

print("The training dataset has %i days, the validation dataset has %i day and the test set has %i days"
      %(len(train_days), len(validation_days), len(test_days)))

x_train = pd.concat([input_data.loc[input_data.index.date == _] for _ in train_days])
y_train = output_data.loc[x_train.index, :]
print("Train set created")

x_validation = pd.concat([input_data.loc[input_data.index.date == _] for _ in validation_days])
y_validation = output_data.loc[x_validation.index, :]
print("Validation set created")

x_test = pd.concat([input_data.loc[input_data.index.date == _] for _ in test_days])
y_test = output_data.loc[x_test.index, :]
print("Test set created")

The training dataset has 438 days, the validation dataset has 74 day and the test set has 219 days
Train set created
Validation set created
Test set created


In [6]:
train_set = DailyTimeSeriesFromPandas(x_train, y_train)
validation_set = DailyTimeSeriesFromPandas(x_validation, y_validation)
test_set = DailyTimeSeriesFromPandas(x_test, y_test)

In [7]:
# Define model
input_dimensions = len(input_data.columns)
hidden_size = 128
max_length = 27
model_name = 'lstm'
num_layers = 2
use_gpu=True
initial_lr = 1e-3
lstm = True
only_eval = False
nb_epoch = 200
batch_size = 64
bidirectional = True
cell_type = 'gru'
learning_rate_decay_fr = 25    
dropout = 0.5
class CustomLSTMModel(nn.Module):
    def __init__(self, 
                 input_dimensions, 
                 hidden_size, 
                 num_layers, 
                 batch_first, 
                 out_lstm_size=2, 
                 use_conv=False,
                 bidirectional=True,
                cell_type='lstm',
                dropout=0.5, max_length=None):
        
        super(CustomLSTMModel, self).__init__()
        
        self.use_conv = use_conv
        self.cell_type = cell_type
        if cell_type=='lstm':
            self.inner_model = nn.LSTM(input_dimensions, hidden_size, 
                                       dropout=dropout, num_layers=num_layers, 
                                       batch_first=batch_first, 
                                       bidirectional=bidirectional)
        if cell_type=='bnlstm':
            bidirectional = False
            self.inner_model = LSTM(BNLSTMCell, input_dimensions, hidden_size, num_layers, 
                                    batch_first=batch_first, dropout=dropout, max_length=max_length)
        
        elif cell_type=='gru':
            self.inner_model = nn.GRU(input_dimensions, hidden_size, 
                                       dropout=dropout, num_layers=num_layers, 
                                       batch_first=batch_first, 
                                       bidirectional=bidirectional)            
        elif cell_type=='rnn':
            self.inner_model = nn.RNN(input_dimensions, hidden_size, 
                                       num_layers=num_layers, 
                                       batch_first=batch_first, dropout=dropout, 
                                      bidirectional=bidirectional)
        
        mult = 2 if bidirectional else 1
        
        if use_conv:
            self.output_model = nn.Sequential(*[nn.Conv1d(hidden_size*mult, hidden_size, kernel_size=3, padding=1),
                                              nn.ReLU(True),
                                              nn.Conv1d(hidden_size, 1, kernel_size=3, padding=1),
                                              nn.ReLU(True)])
        else:
            
            self.output_model = nn.RNN(hidden_size*mult, out_lstm_size, 
                                       num_layers=1, 
                                       batch_first=batch_first,
                                       nonlinearity='relu')

    def forward(self, x):
        lstm_out = self.inner_model(x)[0]
        if not self.use_conv:
            if self.cell_type=="bnlstm":
                lstm_out = lstm_out.transpose(0,1)
            out, _ = self.output_model(lstm_out)
        else:
            lstm_out = lstm_out.transpose(1,2)
            out = self.output_model(lstm_out)
            out = out.transpose(1,2)
        return out
    
class CustomFCModel(nn.Module):
    def __init__(self, input_dimensions, hidden_size, num_layers=3):
        super(CustomFCModel, self).__init__()
        
        
        model = [nn.Conv1d(input_dimensions, hidden_size, kernel_size=1, padding=0), nn.ReLU(True)]
        for i in range(num_layers-1):
            model.append(nn.Conv1d(hidden_size//(2**i), hidden_size//(2**(i+1)), kernel_size=1))
            model.append(nn.ReLU(True))
            
        model += [nn.Conv1d(hidden_size//(2**(num_layers-1)), 2, kernel_size=1), nn.ReLU(True)]
        self.model = nn.Sequential(*model)
                                              
    def forward(self, x):
        x = x.transpose(1,2)
        out = self.model(x)
        return out.transpose(1,2)
        


In [8]:
def adjust_learning_rate(optimizer, epoch):
        """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
        lr = initial_lr * (0.5 ** (epoch // learning_rate_decay_fr))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr  
def train(model, initial_lr, train_set, validation_set, nb_epoch, experience):
    
    output_dir = "checkpoints/"+experience+"/"
    if not exists(output_dir):
        makedirs(output_dir)
        
    criterion = nn.MSELoss()
    model.cuda()
    params = model.parameters()
    optimizer = optim.Adam(params=params, lr=initial_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-4)
    validation_losses = []
    train_losses = []
    best_model_path = ""
    with Process(experience, nb_epoch) as p:
        p.start()
        for iter_epoch in range(nb_epoch):
            train_loader = DataLoader(dataset=train_set,
                                        batch_size=batch_size, 
                                      shuffle=True, pin_memory=True)
            p.update(1)
            for i, train_batch in enumerate(train_loader):
                train_data, train_label = train_batch
                if use_gpu:
                    train_data = train_data.cuda()
                    train_label = train_label.cuda()

                model.train(True)
                optimizer.zero_grad()
                out = model(train_data)
                train_loss = criterion(out[:,:,0], train_label[:,:,0])+criterion(out[:,:,1], train_label[:,:,1])
                train_loss.backward()
                clip_grad_norm_(parameters=params, max_norm=0.25)
                optimizer.step()


                if iter_epoch % learning_rate_decay_fr ==0:
                    adjust_learning_rate(optimizer, iter_epoch)



                if i % 5 == 0:
                    valid_loader = DataLoader(dataset=validation_set,
                                        batch_size=batch_size, shuffle=True, pin_memory=True)
                    valid_loss = []
                    with torch.no_grad():
                        for valid_batch in valid_loader:
                            valid_data, valid_label = valid_batch
                            if use_gpu:
                                valid_data = valid_data.cuda()
                                valid_label = valid_label.cuda()

                            model.train(False)
                            pred_valid = model(valid_data)
                            loss = criterion(pred_valid[:,:,0], valid_label[:,:,0]) + criterion(pred_valid[:,:,1], valid_label[:,:,1])
                            valid_loss.append(loss.cpu().numpy())
                    mean_loss = np.mean(valid_loss)
                    log.info("Epoch %i/%i, iteration %i, validation loss %.5f, training loss %.5f" 
                          %(iter_epoch, nb_epoch, i,  mean_loss, train_loss))

                    if iter_epoch and i and mean_loss < np.min(validation_losses):

                        best_model_path = join(output_dir, "e_%i_i_%i_l_%.5f.torch"%(iter_epoch, i, mean_loss))
                        torch.save(model.state_dict(), best_model_path)

                    train_losses.append(float(train_loss))
                    validation_losses.append(mean_loss)
        p.succeed()
                
    return train_losses, validation_losses, best_model_path

In [9]:
results = {}

# exp = "bilstm"
# model = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="lstm", dropout=dropout)
# t_l, v_l, best_model_path = train(model, initial_lr, train_set, validation_set, nb_epoch, exp)
# results[exp] = dict(train=t_l, validation=v_l, best_model_path=best_model_path)

exp = "bigru_L1_train"
model = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
                        batch_first=True, out_lstm_size=2, 
                        use_conv=False,
                        bidirectional=True,
                       cell_type="gru", dropout=dropout)
t_l, v_l, best_model_path = train(model, initial_lr, train_set, validation_set, nb_epoch, exp)
results[exp] = dict(train=t_l, validation=v_l, best_model_path=best_model_path)

# exp = "birnn"
# model = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="rnn", dropout=dropout)
# t_l, v_l, best_model_path = train(model, initial_lr, train_set, validation_set, nb_epoch, exp)
# results[exp] = dict(train=t_l, validation=v_l, best_model_path=best_model_path)

# exp = "fullyConvolutional"
# model = CustomFCModel(input_dimensions, hidden_size=256, num_layers=3, dropout=dropout)
# t_l, v_l, best_model_path = train(model, initial_lr, train_set, validation_set, nb_epoch, exp)
# results[exp] = dict(train=t_l, validation=v_l, best_model_path=best_model_path)


# exp = "bnlstm"
# model = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=False,
#                        cell_type="bnlstm", max_length=max_length, dropout=dropout)
# t_l, v_l, best_model_path = train(model, initial_lr, train_set, validation_set, nb_epoch, exp)
# results[exp] = dict(train=t_l, validation=v_l, best_model_path=best_model_path)


# exp = "large-bigru"
# large_hidden_size = hidden_size*2
# large_num_layers = 3
# model = CustomLSTMModel(input_dimensions, large_hidden_size, large_num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="gru", dropout=dropout)
# t_l, v_l, best_model_path = train(model, initial_lr, train_set, validation_set, nb_epoch, exp)
# results[exp] = dict(train=t_l, validation=v_l, best_model_path=best_model_path)




In [17]:
def get_best(folder):
    list_files = listdir(folder)
    list_files = [_ for _ in list_files if _ != ".directory"]
    losses = [float(_.split('l_')[1].replace('.torch', '')) for _ in list_files]
    argmin = np.argmin(losses)
    return join(folder, list_files[argmin])

results = {}
exp = "fullyConvolutional"    
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

exp = "bilstm"    
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

exp = "bigru"    
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

exp = "bigru_L1_train"    
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

exp = "birnn"    
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

exp = "bnlstm"    
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

exp = "large-bigru"
path = join("checkpoints", exp)
path = get_best(path)
results[exp] = dict(best_model_path=path)

In [23]:
models = []

# exp = "fullyConvolutional"
# model_fullyConvolutional = CustomFCModel(211, hidden_size=256, num_layers=3)   
# model_fullyConvolutional.load_state_dict(torch.load(results[exp]['best_model_path']))
# models.append((model_fullyConvolutional, exp))

# exp = "bilstm"
# model_lstm = CustomLSTMModel(211, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="lstm")
# model_lstm.load_state_dict(torch.load(results[exp]['best_model_path']))
# models.append((model_lstm, exp))

exp = "bigru"
model_gru = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
                        batch_first=True, out_lstm_size=2, 
                        use_conv=False,
                        bidirectional=True,
                       cell_type="gru")
model_gru.load_state_dict(torch.load(results[exp]['best_model_path']))
models.append((model_gru, exp))


# exp = "bigru"
# model_gru = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="gru")
# model_gru.load_state_dict(torch.load(results[exp]['best_model_path']))
# models.append((model_gru, exp))


# exp = "birnn"
# model_rnn = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="rnn")
# model_rnn.load_state_dict(torch.load(results[exp]['best_model_path']))
# models.append((model_rnn, exp))


# exp = "bnlstm"
# model_bnlstm = CustomLSTMModel(input_dimensions, hidden_size, num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=False,
#                        cell_type="bnlstm", max_length=max_length)
# model_bnlstm.load_state_dict(torch.load(results[exp]['best_model_path']))
# models.append((model_bnlstm, exp))

# exp = "large-bigru"
# large_hidden_size = hidden_size*2
# large_num_layers = 3
# model_large_bigru = CustomLSTMModel(input_dimensions, large_hidden_size, large_num_layers, 
#                         batch_first=True, out_lstm_size=2, 
#                         use_conv=False,
#                         bidirectional=True,
#                        cell_type="gru", dropout=dropout)
# model_large_bigru.load_state_dict(torch.load(results[exp]['best_model_path']))

# models.append((model_large_bigru, exp))



In [52]:
%matplotlib inline
import matplotlib.pyplot as plt
plot = False
save_res = True
use_uncertainty = True
def plot_series(pred, true):
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
    shape = pred.shape
    pred = pred.reshape(((np.prod(shape[:2]),)+shape[2:]))
    true = true.reshape(((np.prod(shape[:2]),)+shape[2:]))
    index =  np.arange(np.prod(shape[:2]))
    
    ax1.plot(index, pred[:,0], label="Prediction-S1", color="blue")
    ax1.plot(index, true[:,0], label="Groundtruth-S1", color="orange")
    
    ax2.plot(index, pred[:,1], label="Prediction-S2", color="green")
    ax2.plot(index, true[:,1], label="Groundtruth-S2", color="red")
    
    f.legend()
    return f


result_folder = "results_plots/"
with torch.no_grad():
    if plot:
        test_loader = DataLoader(dataset=test_set,
                         batch_size=10, 
                         shuffle=False, 
                         pin_memory=True)

        for iter_batch, test_batch in enumerate(test_loader):
            test_data, test_label = test_batch
            if use_gpu:
                test_data = test_data.cuda()
            for model in models:
                if use_gpu:
                    model[0].cuda()
                f = plot_series(model[0](test_data).cpu().numpy(), test_label.numpy())
                f.suptitle(model[1], fontsize=16)
                f.set_size_inches(14, 6)
                path = join(result_folder, model[1]+"/")
                if not exists(path):
                    makedirs(path)
                f.show()
                plt.savefig(join(path, "iter_%i"%iter_batch+".png"), bbox_inches='tight')
    if save_res:
        dict_results = {}
        
        test_loader = DataLoader(dataset=test_set,
                         batch_size=len(test_set), 
                         shuffle=False, 
                         pin_memory=True)
        for iter_batch, test_batch in enumerate(test_loader):
            test_data, test_label = test_batch
            if use_gpu:
                    test_data = test_data.cuda()
            for model, name in models:
                if use_gpu:
                    model.cuda()
                model.eval()
                if use_uncertainty:
                    model.train()
                    s1_ = []
                    s2_ = []
                    for i in range(500):
                        torch.manual_seed(i)
                        s1_.append(model(test_data)[:,:, 0].cpu().numpy().flatten())
                        s2_.append(model(test_data)[:,:, 1].cpu().numpy().flatten())
                    s1 = np.mean(s1_, axis=0)
                    s2 = np.mean(s2_, axis=0)
                    
                    dict_results['uncertain_'+name+'_s1'] = np.std(s1_, axis=0)
                    dict_results['uncertain_'+name+'_s2'] = np.std(s2_, axis=0)
                    

                dict_results['s1_'+name] = s1
                dict_results['s2_'+name] = s2


                

In [53]:
result = pd.concat([pd.DataFrame(data=dict_results, index=x_test.index), y_test], axis='columns')
result = result.dropna(axis="rows", how="any") # Remove columns that contain NaN


In [54]:
def get_result(df_result):
    def per_columns(given_col):
        s1 = df_result[given_col].values
        s1[np.isnan(s1)]=0
        l1_norm = np.mean(np.abs(s1))
        results_dict = {}
        for col in df_result.columns:
            if col.startswith(given_col+'_'):
                model = col[3:]
                pred = df_result[col].values
                l2_loss = np.sqrt(np.mean((pred-s1)**2))
                results_dict[model] = 100*l2_loss/l1_norm
        return results_dict
    
    print(10*"*", "s1", 10*"*")
    s1_perf = per_columns('s1')
    for key in s1_perf:
        print(key, ":", s1_perf[key])
    
    print(10*"*", "s2", 10*"*")
    s1_perf = per_columns('s2')
    for key in s1_perf:
        print(key, ":", s1_perf[key])
    
                        
get_result(result)

********** s1 **********
bigru : 56.97827243572545
********** s2 **********
bigru : 44.2548129718453


In [44]:
result

,uncertain_bigru_s1,uncertain_bigru_s2,s1_bigru,s2_bigru,s1,s2
2017-06-13 06:00:00,4.231309,2.183085,31.110546,16.954218,50.8,30.2
2017-06-13 06:30:00,5.285409,3.114902,59.070312,34.564999,79.7,47.9
2017-06-13 07:00:00,5.386156,3.385872,88.937836,53.709023,98.7,64.1
2017-06-13 07:30:00,3.461663,3.192396,112.425392,71.369843,118.2,78.6
2017-06-13 08:00:00,1.425358,2.645979,124.371330,83.936394,129.7,91.1
2017-06-13 08:30:00,0.606639,2.288837,129.262115,91.889542,154.3,101.2
2017-06-13 09:00:00,0.417288,2.032220,131.435791,97.125969,166.4,107.8
2017-06-13 09:30:00,0.325383,1.778793,132.654831,100.532471,175.9,111.4
2017-06-13 10:00:00,0.309117,1.579206,133.449936,102.583374,186.4,112.9
2017-06-13 10:30:00,0.279882,1.542879,134.014297,103.474564,192.4,113.2
